# **0. Предобработка данных**

Список загружаемых библиотек:

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import plotly.express as px
import plotly.graph_objects as go
from tabulate import tabulate
# чтобы не получать лишние предупреждения
pd.options.mode.chained_assignment = None

Загрузка доступа к gdrive для доступа к файлу данных.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
df = pd.read_json('/gdrive/MyDrive/moskva.geojson')

Так как мы работаем с форматом geodrive, то нам понадобится загружать строки большей длины. Для этого увеличим число выводимых символов.

In [ ]:
pd.options.display.max_colwidth = 5000

Выведем первые строки файла.

In [ ]:
df.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [37.412325, 55.961122]}, 'properties': {'id': 1563301, 'gibdd_id': '195406532', 'point': {'lat': 55.961122, 'long': 37.412325}, 'participant_categories': ['Пешеходы', 'Все участники'], 'severity': 'Тяжёлый', 'region': 'Молжаниновский', 'parent_region': 'Москва', 'datetime': '2016-07-11 21:20:00', 'address': 'г Москва, ш Международное', 'participants_count': 2, 'injured_count': 1, 'dead_count': 0, 'category': 'Наезд на пешехода', 'light': 'Светлое время суток', 'nearby': [], 'weather': ['Ясно'], 'road_conditions': ['Сухое'], 'vehicles': [{'brand': 'ВАЗ', 'model': 'Kalina', 'color': 'Серый', 'year': 2011, 'category': 'А-класс (особо малый) до 3,5 м', 'participants': [{'health_status': 'Не пострадал', 'role': 'Водитель', 'gender': 'Мужской', 'years_of_driving_experience': 6, 'violations': []}]}], 'participants': [{'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении', 'role': 'Пешеход', 'gender': 'Мужской', 'violations': ['Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода']}], 'tags': ['Дорожно-транспортные происшествия'], 'scheme': '820'}}"
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [37.606, 55.883]}, 'properties': {'id': 1529678, 'gibdd_id': '192559706', 'point': {'lat': 55.883, 'long': 37.606}, 'participant_categories': ['Пешеходы', 'Все участники'], 'severity': 'Легкий', 'region': 'Бибирево', 'parent_region': 'Москва', 'datetime': '2016-04-14 19:10:00', 'address': 'г Москва, ул Плещеева, 4', 'participants_count': 3, 'injured_count': 2, 'dead_count': 0, 'category': 'Наезд на пешехода', 'light': 'Светлое время суток', 'nearby': ['Многоквартирные жилые дома', 'Остановка общественного транспорта'], 'weather': ['Ясно'], 'road_conditions': ['Сухое'], 'vehicles': [{'brand': 'SKODA', 'model': 'Octavia', 'color': 'Серый', 'year': 2012, 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'health_status': 'Не пострадал', 'role': 'Водитель', 'gender': 'Женский', 'years_of_driving_experience': 16, 'violations': []}]}], 'participants': [{'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'role': 'Пешеход', 'gender': 'Женский', 'violations': ['Переход через проезжую часть в неустановленном месте (при наличии в зоне видимости перекрёстка)']}, {'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'role': 'Пешеход', 'gender': 'Мужской', 'violations': ['Переход через проезжую часть в неустановленном месте (при наличии в зоне видимости перекрёстка)']}], 'tags': ['Дорожно-транспортные происшествия'], 'scheme': '820'}}"
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [37.777691, 55.763374]}, 'properties': {'id': 1528379, 'gibdd_id': '217249072', 'point': {'lat': 55.763374, 'long': 37.777691}, 'participant_categories': ['Все участники'], 'severity': 'Легкий', 'region': 'Перово', 'parent_region': 'Москва', 'datetime': '2019-03-19 17:00:00', 'address': 'г Москва, ш Энтузиастов, 64', 'participants_count': 3, 'injured_count': 1, 'dead_count': 0, 'category': 'Столкновение', 'light': 'Светлое время суток', 'nearby': ['Зоны отдыха', 'Остановка общественного транспорта', 'Регулируемый перекрёсток', 'Надземный пешеходный переход'], 'weather': ['Ясно'], 'road_conditions': ['Мокрое'], 'vehicles': [{'brand': 'TOYOTA', 'model': 'Avensis', 'color': 'Зеленый', 'year': 2000, 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении,

# **1. Обработка столбца features и разбиение информации на столбцы.**

Так как основная информация лежит в виде словаря в столбце features, то вытащим ее от туда.

In [ ]:
data_all = pd.DataFrame.from_records([i['properties'] for i in list(df['features'])])

Выведем описание полученного датафрейма:

In [ ]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57130 entries, 0 to 57129
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      57130 non-null  int64 
 1   gibdd_id                57130 non-null  object
 2   point                   57130 non-null  object
 3   participant_categories  57130 non-null  object
 4   severity                57130 non-null  object
 5   region                  57130 non-null  object
 6   parent_region           57130 non-null  object
 7   datetime                57130 non-null  object
 8   address                 53931 non-null  object
 9   participants_count      57130 non-null  int64 
 10  injured_count           57130 non-null  int64 
 11  dead_count              57130 non-null  int64 
 12  category                57130 non-null  object
 13  light                   57130 non-null  object
 14  nearby                  57130 non-null  object
 15  we

Таким образом мы получили 21 столбец, из которых два все еще представляют из себя словарь. Также столбец даты надо перевести в формат datetime. Добавим указание дня недели и часа аварии.

In [ ]:
data_all['datetime'] = pd.to_datetime(data_all['datetime'])
data_all['weekday'] = data_all['datetime'].dt.day_name()
data_all['hour'] = data_all['datetime'].dt.hour

Таким образом полученная таблица данных выглядит как:

In [ ]:
data_all.head(1)

,id,gibdd_id,point,participant_categories,severity,region,parent_region,datetime,address,participants_count,injured_count,dead_count,category,light,nearby,weather,road_conditions,vehicles,participants,tags,scheme,weekday,hour
0,1563301,195406532,"{'lat': 55.961122, 'long': 37.412325}","[Пешеходы, Все участники]",Тяжёлый,Молжаниновский,Москва,2016-07-11 21:20:00,"г Москва, ш Международное",2,1,0,Наезд на пешехода,Светлое время суток,[],[Ясно],[Сухое],"[{'brand': 'ВАЗ', 'model': 'Kalina', 'color': 'Серый', 'year': 2011, 'category': 'А-класс (особо малый) до 3,5 м', 'participants': [{'health_status': 'Не пострадал', 'role': 'Водитель', 'gender': 'Мужской', 'years_of_driving_experience': 6, 'violations': []}]}]","[{'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении', 'role': 'Пешеход', 'gender': 'Мужской', 'violations': ['Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода']}]",[Дорожно-транспортные происшествия],820,Monday,21


Так как у нас есть числовые столбцы, то посмотрим кратко на их распределение:

In [ ]:
data_all.describe()

,id,participants_count,injured_count,dead_count,hour
count,5.713000e+04,57130.000000,57130.000000,57130.000000,57130.000000
mean,1.647731e+06,2.417574,1.141694,0.054140,13.728374
std,2.880663e+05,0.921676,0.630521,0.249052,6.071685
min,1.527476e+06,1.000000,0.000000,0.000000,0.000000
25%,1.541759e+06,2.000000,1.000000,0.000000,9.000000
50%,1.556044e+06,2.000000,1.000000,0.000000,15.000000
75%,1.570326e+06,3.000000,1.000000,0.000000,19.000000
max,2.562158e+06,21.000000,19.000000,8.000000,23.000000


Выводы:

1. Число участников варьируется от 1 до 3 в 75% случаев, максимальное число участников: 21
2. Число раненых составляет 1 человека в 75% случаев, максимальное число: 19
3. Число смертельных исходов в 75% случаев составляет 0, максимальное число смертей: 8

# **2. Используемые функции в анализе.**

Так как часть столбцов представляет из себя список из элементов, то создадим несколько функций для выделения сводной информации по ее параметрам.

1. Создание листа из уникальных значений для столбца title в датафрейме datafr.

In [ ]:
def unique_el(title_column, datafr):
  name = []
  for row in range(len(datafr)):
    for el in datafr.loc[row, title_column]:
      if el not in name:
        name.append(el)
  return name

2. Создание словаря с частотой упоминания уникальных значений для столбца title в датафрейме datafr.

In [ ]:
def uniq_count(title_column, datafr):
  temp_dict = {}
  temp_uniq = unique_el(title_column, datafr)
  for el in temp_uniq:
    count = 0
    for row in range(len(datafr)):
      for element in datafr.loc[row, title_column]:
        if element == el:
          count += 1
      temp_dict[el] = count
  return temp_dict

3. Создание таблицы с частотой появления уникальных значений для столбца title в датафрейме datafr.

In [ ]:
def uniq_table(title_column, datafr):
  uniq_temp = unique_el(title_column, datafr)
  data_temp = pd.DataFrame()
  for el in uniq_temp:
    data_temp['id'] = datafr['id']
    data_temp[el] = [1
                                 if el in datafr.loc[row, title_column]
                                 else 0
                            for row in range(len(datafr))]
  return data_temp

4. Создание сводной таблицы по пострадавшим и выбранному столбцу title в датафрейме datafr.

In [ ]:
def uniq_pivot(title_column, datafr):
  table_temp = pd.pivot_table(datafr, index=[title_column, 'severity'],
                                      values=['injured_count',
                                              'participants_count',
                                              'dead_count'], aggfunc='sum'
                                              ).reset_index()
  table_temp['percent_injured'] = round(table_temp['injured_count'
                                                    ] / table_temp['participants_count'
                                                                  ],2)
  table_temp['percent_dead'] = round(table_temp['dead_count'
                                                ] / table_temp['participants_count'
                                                                ],2)
  return table_temp

# **3. План анализа данных**

Основная интересующая нас информация:

Это зависимость числа пострадавших от того или иного параметра.
Это позволяет разбить задачу по анализу данных на уровни сложности:

1. Анализ числа аварий для различных дней недели, времени происшествия, района и т.д. То есть для столбцов представляенных в виде одного значения.
2. Анализ столбцов представленных листами, т.е. определение связи с погодными условиями, дорожными условиями, типами участников и т.д.
3. Анализ столбцов представленными в виде словаря и построение многопараметрического анализа.

**В работе представлены разборы нескольких случаев для каждого из типов задач.**

* Раздел 4 для простых задач
* Раздел 5 для второго типа
* Раздел 6 для разбора столбцов из словарей


# **4.1 Анализ числа аварий по дням недели.**

Воспользуемся заготовленными функциями и создадим сводную таблицу для данных по дням недели.

In [ ]:
table_weekday = uniq_pivot('weekday', data_all)

In [ ]:
fig = px.bar(table_weekday, x='weekday', y='percent_injured', color='severity')
fig.update_layout(barmode='group')
fig.show()

In [ ]:
px.bar(table_weekday, x='weekday', y='percent_dead')

# **4.2 Анализ числа аварий по времени.**

Воспользуемся заготовленными функциями и создадим сводную таблицу для данных по часам.

In [ ]:
table_time = uniq_pivot('hour', data_all)

In [ ]:
fig = px.bar(table_time, x='hour', y='percent_injured', color='severity')
fig.update_layout(barmode='group')
fig.show()

Выводы:
1. В 5 утра выделяется пик количества аварий.
2. С 9 вечера до 6 утра выделяется резкое увеличение числа аварий, что можно связать с наступлением темноты, а также к усталостью водителя.
3. Аварии с погибшими чаще всего происходят в 5-6 утра.

In [ ]:
px.bar(table_time, x='hour', y='dead_count', color='severity')

Выводы:
1. Количество смертей имеет пик в 12 ночи.
2. С 3 утра до 16 вечера в целом число смертей выходит на плато и не меняется.
3. С 17 до 23 часов выделяется повышенное число смертей, что может быть связано с усталостью водителя или с освещением.


# **4.3 Анализ данных по районам.**

Так как число регионов в таблице большое, то ограничимся топ 10 регионов по числу участников в авариях.

In [ ]:
table_region = uniq_pivot('region', data_all)
table_region_short = table_region.sort_values(by='participants_count', 
                                              ascending=False).head(10)
table_region_fin = table_region[table_region['region'].isin(list(table_region_short.region))]

In [ ]:
fig = px.bar(table_region_fin, x='region', y='percent_injured', color='severity')
fig.update_layout(barmode='group')
fig.show()

Выводы:
1. В Гольяново и Выхино-Жулебино больше всего участников дтп. 
2. Легкие дтп в целом равномерно распределены по районам.
3. С погибшими также чаще всего происходят в Гольяново и Выхино-Жулебино.

In [ ]:
px.bar(table_region_fin, x='region', y='percent_dead', color='severity')

Вывод:

Основной процент погибших приходится на Хорошево-Мневники и Пресненский от всех участников.

# **5.1 Анализ столбца 'participant_categories'**

Обратимся к известным функциям:

In [ ]:
unique_pc = unique_el('participant_categories', data_all)
dict_pc = uniq_count('participant_categories', data_all)

В полученном словаре видно, что параметр все участники появляется в каждой строчке и таким образом не несет никакой информации. 

In [ ]:
dict_pc

{'Велосипедисты': 1669,
 'Все участники': 57130,
 'Дети': 4746,
 'Мотоциклисты': 4784,
 'Общ. транспорт': 1075,
 'Пешеходы': 21102}

Заменим столбцы, где только записано 'Все участники' на 'Только водители'.

In [ ]:
data_all['participant_categories'] = [['Только водители']
                                            if data_all.loc[row,'participant_categories'] == ['Все участники']
                                      else data_all.loc[row,'participant_categories']
                                          for row in range(len(data_all))]

После этого заново создадим словарь с подсчетом встречающихся уникальных значений участников дтп, и удалим из него упоминание 'Все участники'.

In [ ]:
unique_pc = unique_el('participant_categories', data_all)
dict_pc = uniq_count('participant_categories', data_all)

In [ ]:
del dict_pc['Все участники']
go.Figure([go.Bar(x=list(dict_pc.keys()), y = np.array(list(dict_pc.values()))/len(data_all))])
#/len(data_all)

Выводы:
1. Основные участники дтп это только водители 47%
2. В 37% дтп участвуют пешеходы.
3. В 8% участвуют дети или мотоциклисты.
4. На общественный транспорт или велосипедистов остается 2 и 3% соответственно.

# **5.2 Анализ влияния погодных условий на дтп.**

Посмотрим на уникальные параметры погоды:

In [ ]:
unique_weather = unique_el('weather', data_all)
dict_weather = uniq_count('weather', data_all)

Создадим словарь с частотой упоминания того или иного описания погоды:

In [ ]:
dict_weather

{'Дождь': 2518,
 'Метель': 17,
 'Пасмурно': 30008,
 'Снегопад': 1379,
 'Температура выше +30С': 6,
 'Температура ниже -30С': 2,
 'Туман': 28,
 'Ураганный ветер': 1,
 'Ясно': 23695}

Представим полученные данные на графике.

In [ ]:
fig = go.Figure([go.Bar(x=list(dict_weather.keys()), y = list(dict_weather.values()))])
fig.show()

Выводы:
1. Основное описание погодных условий, это или пасмурно или ясно. 
2.  Для погодных условий "Пасмурно" существует уточнение и разделение на дождь или снег. Таких событий всего 2518 и 1379 соответственно.
3. Остальные уточнения важны, но данных по ним очень мало и не несут статистическую значимость(их меньше 1% от всех событий).
4. Заменим все уточнения на: ясно, пасмурно или остальное.

In [ ]:
data_all['weather'] = ['Ясно'
                       if 'Ясно' in list(data_all.loc[row, 'weather'])
                       else 'Пасмурно'
                       if 'Пасмурно' in list(data_all.loc[row, 'weather'])
                       else 'Остальное'
                       for row in range(len(data_all))]


Таким образом мы получили только три уникальных варианта погодных условий: 'Ясно', 'Пасмурно', 'Остальное'. Частота с которой встречаются дтп при этих условиях:

In [ ]:
data_all.weather.value_counts()

Пасмурно     30008
Ясно         23695
Остальное     3427
Name: weather, dtype: int64

Создадим сводную таблицу и посмотрим на распределение типов дтп при каждом из таких погодных условий.

In [ ]:
table_weather_people = uniq_pivot('weather', data_all)
table_weather_people

,weather,severity,dead_count,injured_count,participants_count,percent_injured,percent_dead
0,Остальное,Легкий,0,2438,5055,0.48,0.00
1,Остальное,С погибшими,227,105,535,0.20,0.42
2,Остальное,Тяжёлый,0,1492,2801,0.53,0.00
3,Пасмурно,Легкий,0,24332,51255,0.47,0.00
4,Пасмурно,С погибшими,1629,657,3720,0.18,0.44
5,Пасмурно,Тяжёлый,0,9317,17668,0.53,0.00
6,Ясно,Легкий,0,17994,38170,0.47,0.00
7,Ясно,С погибшими,1237,488,2861,0.17,0.43
8,Ясно,Тяжёлый,0,8402,16051,0.52,0.00


In [ ]:
fig = px.bar(table_weather_people, x='weather', y='percent_injured', color='severity')
fig.update_layout(barmode='group')
fig.show()

Выводы:
* В остальные погодные условия происходит больший процент всех событий, однако число событий меньше.
* В пасмурную погоду происходит больше тяжелых дтп и со смертельным исходом.
* Процент погибших больше в пасмурную погоду, чем в остальные погодные условия. Видно из графика ниже.

In [ ]:
px.bar(table_weather_people, x='weather', y='percent_dead', color='severity')

# **5.3 Анализ влияния дорожных условий на дтп.**

Воспользуемся заготовкйо функций и посмотрим на количество упоминаний уникальных значений в столбце дорожные условия.

In [ ]:
unique_rc = unique_el('road_conditions', data_all)
dict_rc = uniq_count('road_conditions', data_all)
dict_rc

{'Гололедица': 118,
 'Дефекты покрытия': 92,
 'Загрязненное': 249,
 'Залитое (покрытое) водой': 4,
 'Заснеженное': 2050,
 'Иные недостатки': 48,
 'Мокрое': 15634,
 'Нарушения в размещении наружной рекламы': 12,
 'Не установлено': 8,
 'Недостатки зимнего содержания': 313,
 'Недостаточное освещение': 16,
 'Неисправное освещение': 69,
 'Неисправность светофора': 216,
 'Неправильное применение, плохая видимость дорожных знаков': 2391,
 'Неровное покрытие': 10,
 'Несоответствие дорожных ограждений предъявляемым требованиям': 33,
 'Несоответствие железнодорожного переезда предъявляемым требованиям': 2,
 'Несоответствие люков смотровых колодцев и ливневой канализации предъявляемым требованиям': 27,
 'Неудовлетворительное состояние обочин': 17,
 'Неудовлетворительное состояние разделительной полосы': 14,
 'Низкие сцепные качества покрытия': 11,
 'Обработанное противогололедными материалами': 4297,
 'Ограничение видимости': 6,
 'Отсутствие временных ТСОД в местах проведения работ': 280,
 'Отсут

Выделим основные дорожные условия, а именно топ 8. Ограничим их по числу таких событий.

In [ ]:
dict_short_rc = {key: value
                 for key,value in dict_rc.items()
                 if value > 890}

Представим полученные значения на графике:

In [ ]:
go.Figure([go.Bar(x=list(dict_short_rc.keys()), y = np.array(list(dict_short_rc.values()))/len(data_all))])

Видно, что основные условия это сухое, мокрое или заснеженное покрытие. Остальное это уточнение видимости знаков, покрытия и т.д. Таким образом оставим только основные дорожные условия:

In [ ]:
data_all['road_conditions'] = ['Сухое'
                       if 'Сухое' in list(data_all.loc[row, 'road_conditions'])
                       else 'Мокрое'
                       if 'Мокрое' in list(data_all.loc[row, 'road_conditions'])
                       else 'Заснеженное'
                       if 'Заснеженное' in list(data_all.loc[row, 'road_conditions'])                       
                       else 'Остальное'
                       for row in range(len(data_all))]

Создадим сводную таблицу по пострадавшим:

In [ ]:
table_rc = uniq_pivot('road_conditions', data_all)
table_rc_short = table_rc.sort_values(by='participants_count', 
                                              ascending=False).head(10)
table_rc_fin = table_rc[table_rc['road_conditions'].isin(list(table_rc_short.road_conditions))]
table_rc_fin

,road_conditions,severity,dead_count,injured_count,participants_count,percent_injured,percent_dead
0,Заснеженное,Легкий,0,1653,3443,0.48,0.00
1,Заснеженное,С погибшими,75,44,189,0.23,0.40
2,Заснеженное,Тяжёлый,0,705,1324,0.53,0.00
3,Мокрое,Легкий,0,12162,25275,0.48,0.00
4,Мокрое,С погибшими,927,370,2114,0.18,0.44
5,Мокрое,Тяжёлый,0,5348,10050,0.53,0.00
6,Остальное,Легкий,0,3917,8171,0.48,0.00
7,Остальное,С погибшими,266,133,632,0.21,0.42
8,Остальное,Тяжёлый,0,1600,3003,0.53,0.00
9,Сухое,Легкий,0,27032,57591,0.47,0.00


Построим данные по каждому из типов аварии:

In [ ]:
fig = px.bar(table_rc_fin, x='road_conditions', y='percent_injured', color='severity')
fig.update_layout(barmode='group')
fig.show()

In [ ]:
px.bar(table_rc_fin, x='road_conditions', y='percent_dead', color='severity')

Выводы:
* Меньше всего тяжелых аварий на сухих дорогах (52% вместо 53%)
* С погибшими аварий больше на заснеженном покрытии 23%, для мокрого и сухого 17-18%.
* Больше всего процент погибших для мокрого и сухого покрытия 44%, тогда как для заснеженного 40%.


# **6. Обработка данных по маркам машин участвующих в дтп**

В столбце 'vehicles' представлены словари для каждой машины участвующей в дтп. Пройдемся по всему списку и создадим словарь для марки машины и типа ранений водителя.

In [ ]:
car_dict = {}
for vehicle_el in data_all['vehicles']:
  for car in vehicle_el:
    if not car_dict.get(car['brand']):
      car_dict[car['brand']] = {}
    for participant in car['participants']:
      if not car_dict[car['brand']].get(participant['health_status']) :
        car_dict[car['brand']][participant['health_status']] = 1
      else:
        car_dict[car['brand']][participant['health_status']] += 10

In [ ]:
car_dict

Типы телесных повреждений делятся на три:
* не пострадал;
* ранен;
* скончался.
Приведем значения в словаре к этим типам.

In [ ]:
car_dict_short = {}
for car in car_dict.keys():
  car_dict_short[car] = {}
  car_dict_short[car]['Скончался'] = 0
  car_dict_short[car]['Раненый'] = 0
  car_dict_short[car]['Не пострадал'] = 0
  car_dict_short[car]['Общее число'] = 0
  for health in car_dict[car].keys():
    car_dict_short[car]['Общее число'] += car_dict[car][health]
    if "Скончался" in str(health):
      car_dict_short[car]['Скончался'] += car_dict[car][health]
    elif 'Получил' in str(health):
      car_dict_short[car]['Раненый'] += car_dict[car][health]
    elif 'Раненый' in str(health):
      car_dict_short[car]['Раненый'] += car_dict[car][health]
    else:
      car_dict_short[car]['Не пострадал'] += car_dict[car][health]


Теперь перестроим словарь в датафрейм

In [ ]:
table_car = pd.DataFrame()
for index, car in enumerate(car_dict_short.keys()):
  table_car.loc[index, 'brand'] = car
  table_car.loc[index, 'Не пострадал'] = car_dict_short[car]['Не пострадал']
  table_car.loc[index, 'Раненый'] = car_dict_short[car]['Раненый']   
  table_car.loc[index, 'Скончался'] = car_dict_short[car]['Скончался']
  table_car.loc[index, 'Общее число'] = car_dict_short[car]['Общее число']  

In [ ]:
table_car.sort_values(by='Общее число', ascending=False)

,brand,Не пострадал,Раненый,Скончался,Общее число
17,HYUNDAI,66932.0,41096.0,1146.0,109174.0
13,KIA,56132.0,36065.0,850.0,93047.0
18,VOLKSWAGEN,42042.0,24885.0,697.0,67624.0
0,ВАЗ,32402.0,31914.0,1413.0,65729.0
15,FORD,36362.0,19365.0,392.0,56119.0
...,...,...,...,...,...
220,ASTRA,1.0,0.0,0.0,1.0
223,SPYKER,0.0,1.0,0.0,1.0
225,SALEEN,0.0,1.0,0.0,1.0
227,PANONIA,0.0,1.0,0.0,1.0


Посмотрим на описание полученных данных:

In [ ]:
table_car.describe()

,Не пострадал,Раненый,Скончался,Общее число
count,260.000000,260.000000,260.000000,260.000000
mean,2653.876923,1700.530769,43.230769,4397.638462
std,8369.004870,5423.502175,157.841206,13766.632895
min,0.000000,0.000000,0.000000,1.000000
25%,1.000000,1.000000,0.000000,2.000000
50%,31.000000,21.000000,0.000000,51.500000
75%,436.000000,335.750000,1.000000,940.750000
max,66932.000000,41096.000000,1413.000000,109174.000000


Выделим первые 10 брэндов машин по общим участникам дтп и построим для них распределение повреждений в ходе дтп.

In [ ]:
table_car_short = table_car.sort_values(by='Общее число', ascending=False).head(10)

In [ ]:
fig = go.Figure(data = [go.Bar(name='Не пострадал', x=table_car_short['brand'], 
                               y=table_car_short['Не пострадал']), 
                        go.Bar(name='Раненый', x=table_car_short['brand'], 
                               y=table_car_short['Раненый']), 
                        go.Bar(name='Скончался', x=table_car_short['brand'], 
                               y=table_car_short['Скончался'])
                        ])
fig.update_layout(barmode='group')
fig.show()

Такой подход не позволяет говорить о том, где больше погибает или больше не получает повреждений. Но показывает марки машин, которые чаще всего попадают в дтп, и распределение количества и степени повреждений в них.

Для этого оставим в таблице только машины с участниками от 30000. И добавим столбцы процентов смертей, пострадавших и непострадавших.

In [ ]:
table_medium = table_car[table_car['Общее число'] > 15000
                         ].sort_values(by='Общее число', ascending=False)
table_medium['percent_inj'] = round(table_medium['Раненый'] / table_medium['Общее число'],2)
table_medium['percent_dead'] = round(table_medium['Скончался'] / table_medium['Общее число'],2)
table_medium

,brand,Не пострадал,Раненый,Скончался,Общее число,percent_inj,percent_dead
17,HYUNDAI,66932.0,41096.0,1146.0,109174.0,0.38,0.01
13,KIA,56132.0,36065.0,850.0,93047.0,0.39,0.01
18,VOLKSWAGEN,42042.0,24885.0,697.0,67624.0,0.37,0.01
0,ВАЗ,32402.0,31914.0,1413.0,65729.0,0.49,0.02
15,FORD,36362.0,19365.0,392.0,56119.0,0.35,0.01
26,MERCEDES,36012.0,17875.0,331.0,54218.0,0.33,0.01
7,ЛИАЗ,26642.0,22844.0,60.0,49546.0,0.46,0.00
1,SKODA,28742.0,18464.0,244.0,47450.0,0.39,0.01
5,NISSAN,30382.0,15835.0,301.0,46518.0,0.34,0.01
2,TOYOTA,30462.0,14415.0,393.0,45270.0,0.32,0.01


Из таблицы сразу можно заметить, что процент смертности для всех одинаков 1%, кроме Хонды 3%, ВАЗ 2% и минимум смертности на ЛИАЗ 0%. 
Для анализа данных по ранениям лучше всего визуализировать данные.

Выведем топ 10 марок машин с наибольшим количеством не пострадавших, как топ 10 безопасных машин и топ 10 машин, где было больше всего смертей.

In [ ]:
px.bar(table_medium.sort_values(by='percent_inj', ascending=False), x='brand', y='percent_inj')

Выводы:
* Хонда, прочие марки ТС, ВАЗ, ЛИАЗ выделяются пиком по травматичности в дтп на фоне остальных марок
* На данных по ГАЗ резко обрывается плавный спад процента ранений в авариях.

# **7. Общие слова по тестовому заданию**

В хо